### GraphAnnDataModule Use case

This is an interactive notebook that allows the user to train their choice of model on different Squidpy datasets. By walking through the notebook step-by-step, the user can see how the GraphAnnDataModule class can be applied to train different types of models. For more information on the example models to choose from, see https://www.biorxiv.org/content/10.1101/2021.07.11.451750v1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import adata2data
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM
from gpu_spatial_graph_pipeline.models.non_linear_ncem import NonLinearNCEM
from gpu_spatial_graph_pipeline.models.graph_embedding import GraphEmbedding

from ipywidgets import widgets

### Choose/Upload dataset

Run the cell below to choose the dataset to train on

In [3]:
dropvals1 = widgets.Dropdown(options=[('custom', 1),('mibitof', 2)], value=1,description="dataset")
dropvals1


Dropdown(description='dataset', options=(('custom', 1), ('mibitof', 2)), value=1)

Run the cell below to choose the learning type for splitting the data

In [4]:
dropvals2= widgets.Dropdown(options=[('nodewise', 1),('graphwise', 2)], value=1,description="learning type")
dropvals2

Dropdown(description='learning type', options=(('nodewise', 1), ('graphwise', 2)), value=1)

Run the cell below to instantiate your datamodule

In [5]:
if dropvals1.value==1:

    raise NotImplementedError


elif dropvals1.value==2:
    adata = sq.datasets.mibitof()
    feature_names=['Cluster','batch']

    def anndata2data(adata):
        return adata2data(adata,feature_names)

    
    #input of datamodule
    num_features=(len(set(adata.obs[feature_names[0]])),len(set(adata.obs[feature_names[1]])))

    num_genes=adata.X.shape[1]

dm = GraphAnnDataModule(adata=adata, adata2data_fn=anndata2data, num_workers = 8, batch_size=40,learning_type=dropvals2.options[dropvals2.value-1][0])

dm.setup()
itr = 2
print(f"Sample of batches from custom datamodule:")
for batch in dm.train_dataloader():
    print(batch)
    itr -= 1
    if itr<0:
        break


Sample of batches from custom datamodule:
DataBatch(x=[269, 11], edge_index=[2, 240], y=[269, 36], batch=[269], ptr=[4], train_mask=[269], val_mask=[269], test_mask=[269], batch_size=40)
DataBatch(x=[275, 11], edge_index=[2, 240], y=[275, 36], batch=[275], ptr=[4], train_mask=[275], val_mask=[275], test_mask=[275], batch_size=40)
DataBatch(x=[273, 11], edge_index=[2, 240], y=[273, 36], batch=[273], ptr=[4], train_mask=[273], val_mask=[273], test_mask=[273], batch_size=40)


## Choose model

Run the cell below to choose the model to train

In [6]:
dropvals_model = widgets.Dropdown(options=[('custom', 1),('linear_ncem', 2),('nonlinear_ncem', 3),('graph_embedding', 4)], value=1,description="model")
dropvals_model

Dropdown(description='model', options=(('custom', 1), ('linear_ncem', 2), ('nonlinear_ncem', 3), ('graph_embed…

Run the cell below to instantiate your datamodule

In [10]:
if dropvals_model.value==1:
    raise NotImplementedError
elif dropvals_model.value==2:
    model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)
elif dropvals_model.value==3:
    model = NonLinearNCEM(in_channels=num_features,encoder_hidden_dims=10,latent_dim=30,decoder_hidden_dims=10,out_channels=num_genes, lr=0.0001,weight_decay=0.000001)
elif dropvals_model.value==4:
    model = GraphEmbedding(num_features=11,latent_dim=30,lr=0.0001,weight_decay=0.000001)

model

LinearNCEM(
  (model_sigma): LinearSpatial(
    (linear): Linear(in_features=75, out_features=36, bias=True)
  )
  (model_mu): LinearSpatial(
    (linear): Linear(in_features=75, out_features=36, bias=True)
  )
  (loss_module): GaussianNLLLoss()
)

Run the cell below to choose accelerator 

In [8]:
dropvals_gpu= widgets.Dropdown(options=[('gpu', 1),('cpu', 2)], value=1,description="accelerator")
dropvals_gpu

Dropdown(description='accelerator', options=(('gpu', 1), ('cpu', 2)), value=1)

Run the cell below to train your model

In [13]:
if dropvals_gpu.value==1:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=10,log_every_n_steps=10)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=10,log_every_n_steps=10)
    
trainer.fit(model,datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 2.7 K 
1 | model_mu    | LinearSpatial   | 2.7 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Run the cell below to test your model

In [14]:
trainer.test(model, datamodule=dm)

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   -0.33789458870887756    │
│       test_r2_score       │    0.10030569946238883    │
└───────────────────────────┴───────────────────────────┘

[{'test_r2_score': 0.10030569946238883, 'test_loss': -0.33789458870887756}]